---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [7]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [102]:
def date_sorter():
    from dateutil.parser import parse
    from datetime import datetime, date, time
    import locale
    
    locale.setlocale(locale.LC_ALL, 'English') #Change locale for month names
    
    #Parse all date formats from file
    regex = '(?:[01]?\d[/\-](?:(?:[012]\d)|(?:3[01])|\d)[/\-]\d{2,4})|(?:(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[ \.\-\,]+(?:\d{1,2}(?:st|nd|rd|th)?[\-, ]+)?\d{2,4})|(?:\d{1,2} (?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov| Dec)[a-z]*[ \.,]+\d{2,4})|(?:\d{1,2}/\d{4})|(?:(?:(?:19)|(?:20))\d{2})'
    dates = df.str.findall(regex)
    
    #Set default date time for dateutil.parser parse function
    default_date = date(1970, 1, 1)
    default_time = time(0, 0)
    default_dt = datetime.combine(default_date, default_time)

    #Loop though Series and create a new Series with uniform format %Y-%m-%d
    temp = []
    for index_val, series_val in dates.iteritems():
        parse_date = parse(series_val[0], default=default_dt, dayfirst=False, yearfirst=False, fuzzy=True)
        temp.append(parse_date.strftime('%Y-%m-%d'))
    
    return pd.Series(temp).rank(method='dense').sub(1).astype(int)

date_sorter()

0      250
1      155
2        2
3       52
4      286
5       91
6       81
7      208
8      164
9        0
10     153
11      46
12     308
13       7
14     216
15     412
16     118
17      65
18     303
19      71
20     306
21      95
22     214
23      58
24     145
25     119
26     207
27      63
28       4
29     173
30     151
31      12
32     179
33     228
34     422
35     135
36      17
37     190
38     169
39     106
40      48
41     181
42     265
43     345
44     123
45     301
46     231
47     261
48     147
49      44
50      53
51     227
52     238
53       3
54     176
55     184
56     273
57      23
58     409
59     124
60     277
61     139
62     236
63     182
64     264
65      84
66     233
67     230
68     178
69     160
70     122
71     198
72      74
73      32
74     115
75      93
76     187
77     314
78     100
79     309
80     110
81      83
82      37
83     332
84       1
85     112
86     253
87     295
88     129
89     205
90     281